In [2]:
import numpy as np
np.__version__

'1.26.4'

In [1]:
import os
import polars as pl

os.chdir("../../../")

# predictables\encoding\src\lagged_mean_encoding\_dynamic_rolling_sum.py
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_sum import (
    DynamicRollingSum,
    dynamic_rolling_sum,
)
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_count import (
    DynamicRollingCount,
)

: 

In [ ]:
date_fmt_str = "%m/%d/%Y"

lf = pl.scan_csv(
    "/app/predictables/encoding/tests/rolling_date_example.csv"
).with_columns(
    [
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("incr_value").cast(pl.Float64).name.keep(),
        pl.col("month_prior")
        .str.to_date(format=date_fmt_str)
        .cast(pl.Date)
        .name.keep(),
        pl.col("year_prior").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("rolling_sum").cast(pl.Float64).name.keep(),
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).alias("cur_date"),
        pl.col("cat1").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
        pl.col("cat2").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
    ]
)
lf.head().collect()

In [ ]:
(
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("rolling_sum")
    .x_name("val")
    .date_col("date")
    .index_col("index")
    .category_cols(["cat1"])
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_SUM")
    .run()
    # .with_row_index()
).filter(pl.col("ROLLING_SUM(val[cat1])[lag:30/win:30]") > 0)

date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,30_days_prior,60_days_prior,360_days_prior,390_days_prior,sum2,sum_cat1,sum_cat2,sum_catBOTH,cur_date,ROLLING_SUM(val[cat1])[lag:30/win:30],ROLLING_SUM(val[cat1])[lag:30/win:30]_right
date,f64,date,date,f64,cat,cat,str,str,str,str,i64,i64,i64,i64,date,f64,f64
2014-03-04,1.0,2014-02-04,2013-02-04,34.0,"""A""","""D""","""2/2/2014""","""1/3/2014""","""3/9/2013""","""2/7/2013""",33,17,8,5,2014-03-04,1.0,1.0
2014-03-05,1.0,2014-02-05,2013-02-05,35.0,"""A""","""D""","""2/3/2014""","""1/4/2014""","""3/10/2013""","""2/8/2013""",34,18,8,5,2014-03-05,3.0,3.0
2014-03-07,1.0,2014-02-07,2013-02-07,37.0,"""A""","""C""","""2/5/2014""","""1/6/2014""","""3/12/2013""","""2/10/2013""",36,19,14,7,2014-03-07,6.0,6.0
2014-03-08,1.0,2014-02-08,2013-02-08,38.0,"""B""","""E""","""2/6/2014""","""1/7/2014""","""3/13/2013""","""2/11/2013""",37,17,14,6,2014-03-08,4.0,4.0
2014-03-09,1.0,2014-02-09,2013-02-09,39.0,"""A""","""D""","""2/7/2014""","""1/8/2014""","""3/14/2013""","""2/12/2013""",38,21,9,5,2014-03-09,17.0,17.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-12-27,1.0,2018-11-27,2017-11-27,365.0,"""B""","""E""","""11/27/2018""","""10/28/2018""","""1/1/2018""","""12/2/2017""",360,182,116,59,2018-12-27,6205.0,6205.0
2018-12-28,1.0,2018-11-28,2017-11-28,365.0,"""B""","""E""","""11/28/2018""","""10/29/2018""","""1/2/2018""","""12/3/2017""",360,181,116,58,2018-12-28,5840.0,5840.0
2018-12-29,1.0,2018-11-29,2017-11-29,365.0,"""A""","""D""","""11/29/2018""","""10/30/2018""","""1/3/2018""","""12/4/2017""",360,179,136,67,2018-12-29,5110.0,5110.0


In [4]:
(
    DynamicRollingCount()
    .lf(
        DynamicRollingSum()
        .lf(lf.with_row_index())
        .x_col("rolling_sum")
        .x_name("numerator")
        .date_col("date")
        .index_col("index")
        .offset(30)
        .window(30)
        .rejoin(True)
        .run()
        .with_row_index()
    )
    .x_name("denominator")
    .date_col("date")
    .index_col("index")
    .offset(30)
    .window(30)
    .rejoin(True)
    .run()
)

TypeError: expected `other` join table to be a LazyFrame, not a 'DataFrame'

In [ ]:
lf = pl.concat(
    [
        lf,
        DynamicRollingSum()
        .lf(lf.with_row_index())
        .x_col("rolling_sum")
        .x_name("numerator")
        .date_col("date")
        .index_col("index")
        .offset(30)
        .window(30)
        # .rejoin(True)
        .run()
        .select([pl.col("numerator")]),
    ],
    how="horizontal",
)

lf = pl.concat(
    [
        lf,
        DynamicRollingCount()
        .lf(lf.with_row_index())
        .date_col("date")
        .index_col("index")
        .offset(30)
        .window(30)
        # .rejoin(True)
        .run()
        .select([pl.col("count")]),
    ],
    how="horizontal",
)

lf.head().collect()

ColumnNotFoundError: numerator

Error originated just after this operation:
 SELECT [col("index"), col("date"), col("ROLLING_SUM(numerator[ALL])")] FROM
   WITH_COLUMNS:
   [col("rolling_value_list").alias("ROLLING_SUM(numerator[ALL])")]
    LEFT JOIN:
    LEFT PLAN ON: [col("index")]
       SELECT [col("index").alias("index")] FROM
        WITH ROW INDEX
           WITH_COLUMNS:
           [col("date").str.strptime([String(raise)]).strict_cast(Date).alias("date"), col("incr_value").strict_cast(Float64).alias("incr_value"), col("month_prior").str.strptime([String(raise)]).strict_cast(Date).alias("month_prior"), col("year_prior").str.strptime([String(raise)]).strict_cast(Date).alias("year_prior"), col("rolling_sum").strict_cast(Float64).alias("rolling_sum"), col("date").str.strptime([String(raise)]).strict_cast(Date).alias("cur_date"), col("cat1").strict_cast(String).strict_cast(Categorical(None, Physical)).alias("cat1"), col("cat2").strict_cast(String).strict_cast(Categorical(None, Phy

In [ ]:
lf.filter((pl.col("numerator") != 0)).collect()

date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,30_days_prior,60_days_prior,360_days_prior,390_days_prior,sum2,sum_cat1,sum_cat2,sum_catBOTH,cur_date,numerator,count
date,f64,date,date,f64,cat,cat,str,str,str,str,i64,i64,i64,i64,date,f64,f64
2014-03-04,1.0,2014-02-04,2013-02-04,34.0,"""A""","""D""","""2/2/2014""","""1/3/2014""","""3/9/2013""","""2/7/2013""",33,17,8,5,2014-03-04,1.0,30.0
2014-03-05,1.0,2014-02-05,2013-02-05,35.0,"""A""","""D""","""2/3/2014""","""1/4/2014""","""3/10/2013""","""2/8/2013""",34,18,8,5,2014-03-05,3.0,30.0
2014-03-06,1.0,2014-02-06,2013-02-06,36.0,"""B""","""E""","""2/4/2014""","""1/5/2014""","""3/11/2013""","""2/9/2013""",35,16,13,6,2014-03-06,6.0,30.0
2014-03-07,1.0,2014-02-07,2013-02-07,37.0,"""A""","""C""","""2/5/2014""","""1/6/2014""","""3/12/2013""","""2/10/2013""",36,19,14,7,2014-03-07,10.0,30.0
2014-03-08,1.0,2014-02-08,2013-02-08,38.0,"""B""","""E""","""2/6/2014""","""1/7/2014""","""3/13/2013""","""2/11/2013""",37,17,14,6,2014-03-08,15.0,30.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-12-27,1.0,2018-11-27,2017-11-27,365.0,"""B""","""E""","""11/27/2018""","""10/28/2018""","""1/1/2018""","""12/2/2017""",360,182,116,59,2018-12-27,10950.0,30.0
2018-12-28,1.0,2018-11-28,2017-11-28,365.0,"""B""","""E""","""11/28/2018""","""10/29/2018""","""1/2/2018""","""12/3/2017""",360,181,116,58,2018-12-28,10950.0,30.0
2018-12-29,1.0,2018-11-29,2017-11-29,365.0,"""A""","""D""","""11/29/2018""","""10/30/2018""","""1/3/2018""","""12/4/2017""",360,179,136,67,2018-12-29,10950.0,30.0


In [ ]:
rolling_sum_df = (
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("incr_value")
    .x_name("test random name")
    .date_col("date")
    .index_col("index")
    .offset(30)
    .window(30)
    .run()
)

print(rolling_sum_df.columns)

['index', 'date', 'test random name']
